In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
import funkcijos as fn
from scipy import stats
from itertools import product

%matplotlib qt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [18]:
league = pd.read_csv('league_league.csv')
matches = pd.read_csv('league_matches.csv')
players = pd.read_csv('league_players.csv')
teams = pd.read_csv('league_teams.csv')
teams2 = pd.read_csv('league_teams2.csv')
elo = pd.read_csv('elo_raking.csv', encoding = 'unicode_escape', on_bad_lines='skip', sep= ';')
teams_elo = pd.merge(teams, elo, how = 'inner',left_on='team_name', right_on='team_name')[['team_name','elo']]
#elo1 = pd.merge(matches, teams_elo, how = 'left', left_on='home_team_name' ,right_on='team_name')
elo1 = pd.merge(h2h.result(), elo, how = 'left', left_on='home_team_name' ,right_on='common_name')
elo2 = pd.merge(elo1, elo, how = 'right', left_on='away_team_name', right_on='common_name')
elo_matches = elo2.sort_values('Game Week')



In [9]:
#def statistic function ant dataframes
class statistic:
    def home_team_table(df): # df = dataframe:teams t = team name
        global home_table
        home_table = df[['team_name', 'matches_played_home', 'wins_home', 'draws_home','losses_home', 'points_per_game_home',
        'goals_scored_home', 'goals_conceded_home', 'goal_difference_home','goals_scored_per_match_home','goals_conceded_per_match']]
        return home_table    

    def team_goals_home(df, t: str, sc: str): # df - dataFrame, t = team name sc - scored/conc
        a = teams[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
        c = a.loc[a['team_name']==t].values
        if sc == 'scored':
            return float(c[0][1])
        if sc == 'conc':
            return float(c[0][2])

    def away_team_table(df): # ds = dataframe t = team name
        global away_table
        away_table = df[['team_name', 'matches_played_away', 'wins_away', 'draws_away','losses_away', 'points_per_game_away',
        'goals_scored_away', 'goals_conceded_away', 'goal_difference_away','goals_scored_per_match_away','goals_conceded_per_away']]
        return away_table    

    def team_goals_away(df, t: str, sc: str): # df - dataFrame, t = team name ha - home/away
        a = teams[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
        c = a.loc[a['team_name']==t].values
        if sc == 'scored':
            return float(c[0][1])
        if sc == 'conc':
            return float(c[0][2])
    # get teams attack and defense points. df = teams, t = team, ha = home/away, sc = scored/conc
    def goals_avg(df, t: str, ha: str, sc: str):
        if ha == 'home':
            if sc == 'scored':
                a = df[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][1])
            if sc == 'conc':
                a = df[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][2])
        if ha == 'away':
            if sc == 'scored':
                a = df[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][1])
            if sc == 'conc':
                a = df[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][2])
# You don't need this now.
class matches:
    def team_home(df, team1:str):
        t1 = df.loc[df['team_name']==team1]
        return t1
    def team_away(df, team2:str):
        t2 = df.loc[df['team_name']==team2]
        return t2       
class attack:
    def factors(dt, ha:str, ): #dt - dataframe, ha-home/away/total
        if ha == 'total':
            dataframe = dt[['team_name','matches_played','wins','draws','losses','points_per_game',
            'goals_scored','goals_conceded',"goal_difference", 'total_goal_count', 'minutes_per_goal_scored',
            'minutes_per_goal_conceded','clean_sheets','btts_count','fts_count','first_team_to_score_count',
            'corners_total','cards_total','average_possession','shots','shots_on_target','shots_off_target',
            'fouls','goals_scored_half_time','goals_conceded_half_time','leading_at_half_time','draw_at_half_time','losing_at_half_time',
            'average_total_goals_per_match','goals_scored_per_match','goals_conceded_per_match','total_goals_per_match_half_time',
            'goals_scored_per_match_half_time','goals_conceded_per_match_half_time','corners_per_match','cards_per_match','xg_for_avg_overall',
            'xg_against_avg_overall']]
            return dataframe
        if ha == 'home':
            dataframe_home = dt[['team_name','matches_played_home','wins_home','draws_home','losses_home','points_per_game_home',
            'goals_scored_home','goals_conceded_home','total_goal_count_home','goal_difference_home','minutes_per_goal_scored_home',
            'minutes_per_goal_conceded_home','clean_sheets_home','btts_count_home','fts_count_home','first_team_to_score_count_home',
            'corners_total_home','cards_total_home','average_possession_home','shots_home','shots_on_target_home','shots_off_target_home',
            'fouls_home','goals_scored_half_time_home','goals_conceded_half_time_home','leading_at_half_time_home','draw_at_half_time_home','losing_at_half_time_home',
            'average_total_goals_per_match_home','goals_scored_per_match_home','goals_conceded_per_match_home','total_goals_per_match_half_time_home',
            'goals_scored_per_match_half_time_home','goals_conceded_per_match_half_time_home','corners_per_match_home','cards_per_match_home','xg_for_avg_home',
            'xg_against_avg_home']]
            return dataframe_home
        if ha == 'away':
            dataframe_away = dt[['team_name','matches_played_away','wins_away','draws_away','losses_away','points_per_game_away',
            'goals_scored_away','goals_conceded_away','total_goal_count_away','goal_difference_away','minutes_per_goal_scored_away',
            'minutes_per_goal_conceded_away','clean_sheets_away','btts_count_away','fts_count_away','first_team_to_score_count_away',
            'corners_total_away','cards_total_away','average_possession_away','shots_away','shots_on_target_away','shots_off_target_away',
            'fouls_away','goals_scored_half_time_away','goals_conceded_half_time_away','leading_at_half_time_away','draw_at_half_time_away','losing_at_half_time_away',
            'average_total_goals_per_match_away','goals_scored_per_match_away','goals_conceded_per_match_away','total_goals_per_match_half_time_away',
            'goals_scored_per_match_half_time_away','goals_conceded_per_match_half_time_away','corners_per_match_away','cards_per_match_away','xg_for_avg_away',
            'xg_against_avg_away']]
            return dataframe_away
class dt: # all data frames
    def season_table(ha:str):
        return league[['name', 'total_matches', 'total_game_week', 'average_goals_per_match', 'btts_percentage', 'clean_sheets_percentage',   
        'home_scored_advantage_percentage', 'home_defence_advantage_percentage', 'home_advantage_percentage', 'average_corners_per_match',
        'total_corners_for_season','average_cards_per_match','total_cards_for_season','xg_avg_per_match']]
   




In [30]:
# draw football pitch
from mplsoccer.pitch import Pitch
pitch = Pitch(pitch_color='grass', line_color='white', stripe=True)
fig, ax = pitch.draw()

In [31]:
# Linear regression. You can canculated all factors to goals, where y = goals, x = factors(corners, yellow cards, shots on the target...)
a = teams['team_name'].values
b = teams['goals_scored'].values
c = teams['corners_total'].values #change key to a you factor
cg = teams['team_name'].values

fig, g = plt.subplots()
slope, intercept, r, p, std_err = stats.linregress(c,b)

def myfunc(m):
    return slope * m + intercept
mymodel = list(map(myfunc, c))
res = stats.linregress(c, b)

g.scatter(c,b)
for i, txt in enumerate(cg):
    g.annotate(txt, (c[i], b[i]))

plt.plot(c, mymodel, color = 'red')
rsquare = r * r
g.annotate(str(rsquare), (304,102)) 


plt.show()


In [13]:
# poisson distribution
# calculate prb

# choice home and away team

home_team = 'Arsenal FC'
away_team = 'Fulham FC'

#calculate xG home and away

home = statistic.goals_avg(teams, home_team, 'home', 'scored'), statistic.goals_avg(teams, home_team, 'home', 'conc')
away = statistic.goals_avg(teams, away_team, 'away', 'scored'), statistic.goals_avg(teams, away_team, 'away', 'conc')

a, b = home
a1, b1 = away

xG_home = (a+b1)/2
xG_away = (a1+b)/2



home_prb = []
for i in np.arange(10):
    home_prb.append(poisson.pmf(k=i, mu=xG_home))

away_prb = []
for i in np.arange(10):
    away_prb.append(poisson.pmf(k=i, mu=xG_away))

prod_table = np.array([(i*j) for i, j in product(home_prb, away_prb)])
prod_table.shape = (10, 10)

prob_df = pd.DataFrame(prod_table, index=range(10), columns=range(10))
    
prob_df.iat[0,0]




0.04855782127000997

In [23]:
class statistic:
    def home_team_table(df): # ds = dataframe t = team name
        global home_table
        home_table = df[['team_name', 'matches_played_home', 'wins_home', 'draws_home','losses_home', 'points_per_game_home',
        'goals_scored_home', 'goals_conceded_home', 'goal_difference_home','goals_scored_per_match_home','goals_conceded_per_match']]
        return home_table    

    def team_goals_home(df, t: str, sc: str): # df - dataFrame, t = team name sc - scored/conc
        a = teams[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
        c = a.loc[a['team_name']==t].values
        if sc == 'scored':
            return float(c[0][1])
        if sc == 'conc':
            return float(c[0][2])

    def away_team_table(df): # ds = dataframe t = team name
        global away_table
        away_table = df[['team_name', 'matches_played_away', 'wins_away', 'draws_away','losses_away', 'points_per_game_away',
        'goals_scored_away', 'goals_conceded_away', 'goal_difference_away','goals_scored_per_match_away','goals_conceded_per_away']]
        return away_table    

    def team_goals_away(df, t: str, sc: str): # df - dataFrame, t = team name ha - home/away
        a = teams[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
        c = a.loc[a['team_name']==t].values
        if sc == 'scored':
            return float(c[0][1])
        if sc == 'conc':
            return float(c[0][2])

    def goals_avg(df, t: str, ha: str, sc: str):
        if ha == 'home':
            if sc == 'scored':
                a = df[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][1])
            if sc == 'conc':
                a = df[['team_name','goals_scored_per_match_home','goals_conceded_per_match_home']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][2])
        if ha == 'away':
            if sc == 'scored':
                a = df[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][1])
            if sc == 'conc':
                a = df[['team_name','goals_scored_per_match_away','goals_conceded_per_match_away']] 
                c = a.loc[a['team_name']==t].values
                return float(c[0][2])
    

In [22]:
#get factors values from dataset
class factor:
    def corners(df, home_team:str, ha):
        if ha == "home":
            corn_home = df.loc[df['team_name']==home_team]           
            stat_home = corn_home['corners_total_home'].values/corn_home['matches_played_home'].values
            return stat_home[0]
        if ha == 'away':
            corn_home = df.loc[df['team_name']==home_team]
            stat_home = corn_home['corners_total_away'].values/corn_home['matches_played_away'].values
            return stat_home[0]

#factor.corners(teams, 'Arsenal FC', 'Fulham FC')
    def home_advantage(df, home_team:str,ha):
        if ha == 'home':
            adv_home = df.loc[df['team_name']==home_team]
            adv_prc = adv_home['home_advantage_percentage'].values
            home_adv = adv_home['matches_played_home'].values/adv_home['goal_difference_home'].values
            return adv_prc[0], home_adv[0]
        if ha == 'away':
            adv_home = df.loc[df['team_name']==home_team]
            adv_prc = adv_home['home_advantage_percentage'].values
            home_adv = adv_home['matches_played_away'].values/adv_home['goal_difference_home'].values
            return adv_prc[0], home_adv[0]
    
    def shots_on_target(df, home_team:str,ha, avg):
        if avg == False:
            if ha == 'home':
                shhome = df.loc[df['team_name']==home_team]
                hm =shhome[['shots_on_target_home']].values
                return hm[0][0]
            if ha == 'away':
                shhome = df.loc[df['team_name']==home_team]
                hm =shhome[['shots_on_target_away']].values
                return hm[0][0]
        if avg == True:
            if ha == 'home':
                shhome = df.loc[df['team_name']==home_team]
                hm =shhome[['shots_on_target_home']].values/shhome['matches_played_home'].values
                return hm[0][0]
            if ha == 'away':
                shhome = df.loc[df['team_name']==home_team]
                hm =shhome[['shots_on_target_away']].values/shhome['matches_played_away'].values
                return hm[0][0]

    def fouls(df, home_team:str, ha, avg):
        if avg == False:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fouls_home"].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fouls_away"].values
                return hm[0]
        if avg == True:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fouls_home"].values/fouls['matches_played_home'].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fouls_away"].values/['matches_played_awa'].values
                return hm[0]

    def clean_sheets(df, home_team:str, ha, avg):
        if avg == False:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["clean_sheets_home"].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["clean_sheets_away"].values
                return hm[0]
        if avg == True:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["clean_sheets_home"].values/fouls['matches_played_home'].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["clean_sheets_away"].values/['matches_played_awa'].values
                return hm[0]

    def cards(df, home_team:str, ha, avg):
        if avg == False:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["cards_total_home"].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["cards_total_away"].values
                return hm[0]
        if avg == True:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["cards_total_home"].values/fouls['matches_played_home'].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["cards_total_away"].values/['matches_played_awa'].values
                return hm[0]

    def fts(df, home_team:str, ha, avg):
        if avg == False:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fts_count_home"].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fts_count_away"].values
                return hm[0]
        if avg == True:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fts_count_home"].values/fouls['matches_played_home'].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["fts_count_away"].values/['matches_played_awa'].values
                return hm[0]
#average_possession_home

    def possesion(df, home_team:str,  ha, avg):
        if avg == False:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["average_possession_home"].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["average_possession_away"].values
                return hm[0]
        if avg == True:
            if ha == 'home':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["average_possession_home"].values/fouls['matches_played_home'].values
                return hm[0]
            if ha == 'away':
                fouls = df.loc[df['team_name']==home_team]
                hm =fouls["average_possession_away"].values/fouls['matches_played_away'].values
                return hm[0]


    






In [24]:
#get time factors from dataset
class game_fact:
    def minutes_scored(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_scored_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_scored_away'].values
            return min_scored[0]
    
    def minutes_conc(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_conceded_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['minutes_per_goal_conceded_away'].values
            return min_scored[0]
    def btts(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['btts_count_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['btts_count_away'].values
            return min_scored[0]
    def half_scored(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_scored_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_scored_half_time_away'].values
            return min_scored[0]
    def half_conc(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_conceded_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['goals_conceded_half_time_away'].values
            return min_scored[0]
    
    def leading_HT(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['leading_at_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['leading_at_half_time_away'].values
            return min_scored[0]

    def draw_HT(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['draw_at_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['draw_at_half_time_away'].values
            return min_scored[0]

    def losses_HT(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['losing_at_half_time_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['losing_at_half_time_away'].values
            return min_scored[0]

    def over_under(df, team:str, ha, over, count):
        if ha == 'home':
            if over == True:
                if count == 0.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over05_count_home'].values 
                    return min_scored[0]
                if count == 1.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over15_count_home'].values 
                    return min_scored[0]
                if count == 2.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over25_count_home'].values 
                    return min_scored[0]
                if count == 3.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over35_count_home'].values 
                    return min_scored[0]
                if count == 4.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over45_count_home'].values 
                    return min_scored[0]
                if count == 5.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over55_count_home'].values 
                    return min_scored[0]

        if ha == 'away':
            if over == False:
                if count == 0.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over05_count_away'].values 
                    return min_scored[0]
                if count == 1.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over15_count_away'].values 
                    return min_scored[0]
                if count == 2.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over25_count_away'].values 
                    return min_scored[0]
                if count == 3.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over35_count_away'].values 
                    return min_scored[0]
                if count == 4.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over45_count_away'].values 
                    return min_scored[0]
                if count == 5.5:
                    scored = df.loc[df['team_name']==team]
                    min_scored = scored['over55_count_away'].values 
                    return min_scored[0]
    
    def firs_score_prc(df, team:str, ha):
        if ha == 'home':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['first_team_to_score_percentage_home'].values 
            return min_scored[0]
        if ha == 'away':
            scored = df.loc[df['team_name']==team]
            min_scored = scored['first_team_to_score_percentage_away'].values
            return min_scored[0]
    

In [25]:
# get h2h and other statistic
class h2h:
    def grp_name(df, ha, team:str):
        home_grp = df.groupby('home_team_name').get_group(team)
        away_grp = df.groupby('away_team_name').get_group(team)
        if ha == True:
            return home_grp
        if ha == False:
            return away_grp
    def referee(df, ha):
        if ha == 'home':
            ref1 = df[['referee','home_team_yellow_cards','home_team_red_cards']]
            ref = ref1.groupby('referee').mean()
            return ref
        if ha == 'away':
            ref1 = df['referee','away_team_yellow_cards','away_team_red_cards' ]
            ref = ref1.groupby('referee').mean()
            return ref
    def result():
        res = matches[['home_team_name','away_team_name','Game Week','home_team_goal_count','away_team_goal_count','total_goal_count']]
        return res
    def week(df):
        a = df[['home_team_name','away_team_name','Game Week','home_team_goal_count','away_team_goal_count','total_goal_count']]
        b = a.groupby('Game Week').sum()
        return b
    def get_week(df, week):
        a = df[['home_team_name','away_team_name','Game Week','home_team_goal_count','away_team_goal_count','total_goal_count']]
        b = a.groupby('Game Week').get_group(week)
        return 
    


In [32]:
#get coeficients
class coeficients:
    def zero_inf(df): # Zero inflated model 
        dataframe = h2h.result()
        results = h2h.result().values
        n = 0
        for i in results:
            if i[3] or i[4] == 0:
                n += 1
            else:
                continue
        avg_home = dataframe['home_team_goal_count'].values.mean()
        avg_away = dataframe['away_team_goal_count'].values.mean()

        home_prb = []
        for i in np.arange(10):
            home_prb.append(poisson.pmf(k=i, mu=avg_home))

        away_prb = []
        for i in np.arange(10):
            away_prb.append(poisson.pmf(k=i, mu=avg_away)) 
        
        prod_table = np.array([(i*j) for i, j in product(home_prb, away_prb)])
        prod_table.shape = (10, 10)

        prob_df = pd.DataFrame(prod_table, index=range(10), columns=range(10))

        zero_result = [prob_df.iat[0,0],prob_df.iat[0,1],prob_df.iat[0,2],prob_df.iat[0,3],prob_df.iat[0,4],prob_df.iat[0,5],prob_df.iat[0,6],prob_df.iat[0,7],prob_df.iat[0,8],
        prob_df.iat[1,0],prob_df.iat[2,0],prob_df.iat[3,0],prob_df.iat[4,0],prob_df.iat[5,0],prob_df.iat[6,0],prob_df.iat[7,0],prob_df.iat[8,0],prob_df.iat[9,0]]
        
        poisson_0 = np.array(zero_result).sum()
        count = len(dataframe['home_team_goal_count'].values)+len(dataframe['away_team_goal_count'].values)
        return poisson_0 - n/count
    def elo_goals(elo, df, home_team:str, away_team:str):
        eloh = elo.loc[elo['team_name']==home_team]
        eloa = elo.loc[elo['team_name']==away_team]
        elo_home = eloh['elo'].values
        elo_away = eloa['elo'].values

        data_frame = h2h.result(df)
        total_avg = data_frame['total_goal_count'].mean()
        elo1 = 1/(10**((elo_away[0]-elo_home[0])/400)+1)
        elo2 = 1/(10**((elo_home[0]-elo_away[0])/400)+1)
        xG_home = elo1 * total_avg
        xG_away = elo2 * total_avg
        return xG_home, xG_away
    
    def draw_coef(home_team:str, away_team:str):
        t1 = elo.loc[elo['team_name']==home_team]
        t2 = elo.loc[elo['team_name']==away_team]
        elo1 = t1['elo'].values
        elo2 = t2['elo'].values
        if abs(elo1-elo2) <= 50:
            return 0.12
        if abs(elo1-elo2)>50 and abs(elo1-elo2) <= 150:
            return 0.1
        if abs(elo1-elo2)>150 and abs(elo1-elo2) <= 250:
            return 0.8
        if abs(elo1-elo2)>250 and abs(elo1-elo2) <= 350:
            return 0.6
        if abs(elo1-elo2)>350:
            return 0.4


        
        


In [20]:
class prb:
    def prb_matrix(df, home_team:str, away_team:str, zero_inf=coeficients.zero_inf(matches), draw_coef=coeficients.draw_coef(home_team,away_team), xG = False):
        home = statistic.goals_avg(df, home_team, 'home', 'scored'), statistic.goals_avg(df, home_team, 'home', 'conc')
        away = statistic.goals_avg(df, away_team, 'away', 'scored'), statistic.goals_avg(df, away_team, 'away', 'conc')
        a, b = home
        a1, b1 = away
        xG_home = (a+b1)/2
        xG_away = (a1+b)/2
        home_prb = []
        for i in np.arange(10):
            home_prb.append(poisson.pmf(k=i, mu=xG_home))

        away_prb = []
        for i in np.arange(10):
            away_prb.append(poisson.pmf(k=i, mu=xG_away))
        
        
        
        #prb1 = [zero_inf+(1-draw_coef)*(1-zero_inf)*home_prb[0], (draw_coef+(1-draw_coef)*(1-zero_inf)*home_prb[1])]
        #lst = home_prb[2::]
        home_prb1 = [(zero_inf+(1-draw_coef)*(1-zero_inf)*home_prb[0]),(draw_coef+(1-draw_coef)*(1-zero_inf)*home_prb[1])]
        lst = home_prb[2::]
        for i in lst:
            home_prb1.append(i*(1-draw_coef)*(1-zero_inf))
        
        away_prb1 = [(zero_inf+(1-draw_coef)*(1-zero_inf)*away_prb[0]),(draw_coef+(1-draw_coef)*(1-zero_inf)*away_prb[1])]
        lst1 = away_prb[2::]
        for i in lst1:
            away_prb1.append(i*(1-draw_coef)*(1-zero_inf))
        

        prod_table = np.array([(i*j) for i, j in product(home_prb1, away_prb1)])
        prod_table.shape = (10, 10)

        prob_df = pd.DataFrame(prod_table, index=range(10), columns=range(10))
        if xG == False:
            return prob_df
        if xG == True:
            return xG_home, xG_away
        
    def prb_chances(df, a, b):
        data = prb.prb_matrix(df, a, b, zero_inf=coeficients.zero_inf(matches),draw_coef=0.1)
        home_win = sum((data.iat[1,0]+data.iat[2,0]+data.iat[3,0]+data.iat[4,0]+data.iat[5,0]+data.iat[6,0]+data.iat[7,0]+data.iat[8,0]+data.iat[9,0],
        data.iat[2,1]+data.iat[3,1]+data.iat[4,1]+data.iat[5,1]+data.iat[6,1]+data.iat[7,1]+data.iat[8,1]+data.iat[9,1],
        data.iat[3,2]+data.iat[4,2]+data.iat[5,2]+data.iat[6,2]+data.iat[7,2]+data.iat[8,2]+data.iat[9,2],
        data.iat[4,3]+data.iat[5,3]+data.iat[6,3]+data.iat[7,3]+data.iat[8,3]+data.iat[9,3],
        data.iat[5,4]+data.iat[6,4]+data.iat[7,4]+data.iat[8,4]+data.iat[9,4],
        data.iat[6,5]+data.iat[7,5]+data.iat[8,5]+data.iat[9,5],
        data.iat[7,6]+data.iat[8,6]+data.iat[9,6],
        data.iat[8,7]+data.iat[9,7]+data.iat[9,8]))*100
        draw = (data.iat[0,0]+data.iat[1,1]+data.iat[2,2]+data.iat[3,3]+data.iat[4,4]+data.iat[5,5]+data.iat[6,6]+data.iat[7,7]+data.iat[8,8]+data.iat[9,9])*100
        away_win = sum((data.iat[0,1]+data.iat[0,2]+data.iat[0,3]+data.iat[0,4]+data.iat[0,5]+data.iat[0,6]+data.iat[0,7]+data.iat[0,8]+data.iat[0,9],
        data.iat[1,2]+data.iat[1,3]+data.iat[1,4]+data.iat[1,5]+data.iat[1,6]+data.iat[1,7]+data.iat[1,8]+data.iat[1,9],
        data.iat[2,3]+data.iat[2,4]+data.iat[2,5]+data.iat[2,6]+data.iat[2,7]+data.iat[2,8]+data.iat[2,9],
        data.iat[3,4]+data.iat[3,5]+data.iat[3,6]+data.iat[3,7]+data.iat[3,8]+data.iat[3,9],
        data.iat[4,5]+data.iat[4,6]+data.iat[4,7]+data.iat[4,8]+data.iat[4,9],
        data.iat[5,6]+data.iat[5,7]+data.iat[5,8]+data.iat[5,9],
        data.iat[6,7]+data.iat[6,8]+data.iat[6,9],
        data.iat[7,8]+data.iat[7,9]+data.iat[8,8]))*100
        return home_win, draw, away_win

In [26]:
x = []
y = []
cg = []


for i in teams['team_name'].values:
    a = statistic.goals_avg(teams, i, ha = 'away',sc= 'scored')
    b = factor.fouls(teams, i, ha = 'away', avg=False)
    y.append(a)
    x.append(b)
    cg.append(i)
    
    
fig, g = plt.subplots()
slope, intercept, r, p, std_err = stats.linregress(x,y)

def myfunc(m):
    return slope * m + intercept
mymodel = list(map(myfunc, x))
res = stats.linregress(x, y)

g.scatter(x,y)
for i, txt in enumerate(cg):
    g.annotate(txt, (x[i], y[i]))

plt.plot(x, mymodel, color = 'red')
rsquare = r * r
g.annotate(str(rsquare), (241,2)) 


plt.show()



In [27]:
df = h2h.result()
n = 0
home_win = []
home_win1 = []
home_win2 = []
home_win3 = []
home_win4 = []

draw = []
draw1 = []
draw2 = []
draw3 = []
draw4 = []

away_win = []
away_win1 = []
away_win2 = []
away_win3 = []
away_win4 = []


for i in elo_matches.values:  
    a = elo_matches.loc[n]
    n += 1
    if a['home_team_goal_count'] == a['away_team_goal_count']:
        if abs(a['elo_x']-a['elo_y']) <= 50:
            draw.append(a)
        if abs(a['elo_x']-a['elo_y']) > 50 and abs(a['elo_x']-a['elo_y']) <= 150:
            draw1.append(a)
        if abs(a['elo_x']-a['elo_y']) > 150 and abs(a['elo_x']-a['elo_y']) <= 250:
            draw2.append(a)
        if abs(a['elo_x']-a['elo_y']) > 250 and abs(a['elo_x']-a['elo_y']) <= 350:
            draw3.append(a)
        if abs(a['elo_x']-a['elo_y']) > 350:
            draw4.append(a)
    if a['home_team_goal_count'] > a['away_team_goal_count']:
        if abs(a['elo_x']-a['elo_y']) <= 50:
            home_win.append(a)
        if abs(a['elo_x']-a['elo_y']) > 50 and abs(a['elo_x']-a['elo_y']) <= 150:
            home_win1.append(a)
        if abs(a['elo_x']-a['elo_y']) > 150 and abs(a['elo_x']-a['elo_y']) <= 250:
            home_win2.append(a)
        if abs(a['elo_x']-a['elo_y']) > 250 and abs(a['elo_x']-a['elo_y']) <= 350:
            home_win3.append(a)
        if abs(a['elo_x']-a['elo_y']) > 350:
            home_win4.append(a)
    if a['home_team_goal_count'] < a['away_team_goal_count']:
        if abs(a['elo_x']-a['elo_y']) <= 50:
            away_win.append(a)
        if abs(a['elo_x']-a['elo_y']) > 50 and abs(a['elo_x']-a['elo_y']) <= 150:
            away_win1.append(a)
        if abs(a['elo_x']-a['elo_y']) > 150 and abs(a['elo_x']-a['elo_y']) <= 250:
            away_win2.append(a)
        if abs(a['elo_x']-a['elo_y']) > 250 and abs(a['elo_x']-a['elo_y']) <= 350:
            away_win3.append(a)
        if abs(a['elo_x']-a['elo_y']) > 350:
           away_win4.append(a)


In [29]:
def draw_elo(home_team:str, away_team:str):
    t1 = elo.loc[elo['team_name']==home_team]
    t2 = elo.loc[elo['team_name']==away_team]
    elo1 = t1['elo'].values
    elo2 = t2['elo'].values
    if abs(elo1-elo2) <= 50:
        return 0.12
    if abs(elo1-elo2)>50 and abs(elo1-elo2) <= 150:
        return 0.1
    if abs(elo1-elo2)>150 and abs(elo1-elo2) <= 250:
        return 0.8
    if abs(elo1-elo2)>250 and abs(elo1-elo2) <= 350:
        return 0.6
    if abs(elo1-elo2)>350:
        return 0.4

draw_elo('Chelsea FC', 'Manchester City FC')

0.1